# Creating Redshift Cluster using the AWS python SDK 

In [1]:
import pandas as pd
import boto3
import json
import configparser
import sys
import os
sys.path.append('..')

### Make sure you have an AWS SECRET & ACCESS KEY
- Create a new IAM user in the AWS account
- Give it `AdministratorAccess`, From `Attach existing policies directly` Tab
- Take note of the access key and secret 
- Edit the file `dwh.cfg` in the same folder as this notebook and fill
<BR>
[AWS]<BR>
KEY= YOUR_AWS_KEY<BR>
SECRET= YOUR_AWS_SECRET<BR>
<BR>
<font color='red'>
    REMINDER: Do not include your AWS access keys in the code when sharing this project!
<font/>


# Load DWH Params from a file

In [17]:
config = configparser.ConfigParser()
config.read('../config/dwh.cfg')

['../config/dwh.cfg']

In [18]:
KEY                    = config.get("PRIVATE", "KEY")
SECRET                 = config.get("PRIVATE", "SECRET")

DWH_CLUSTER_TYPE = config.get("NODE", "DWH_CLUSTER_TYPE")
DWH_NUM_NODES    = config.get("NODE", "DWH_NUM_NODES")
DWH_NODE_TYPE    = config.get("NODE", "DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("NODE", "DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("CLUSTER", "DB_NAME")
DWH_DB_USER            = config.get("CLUSTER", "DB_USER")
DWH_DB_PASSWORD        = config.get("CLUSTER", "DB_PASSWORD")
DWH_PORT               = config.get("CLUSTER", "DB_PORT")

REGION_NAME            = config.get("NODE", "REGION_NAME")

DWH_IAM_ROLE_NAME      = config.get("NODE", "DWH_IAM_ROLE_NAME")
(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dar1en-dwh-cluster-dev
4,DWH_DB,dar1en-redshift-dwh-db
5,DWH_DB_USER,dev-dar1en
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dar1en-role-iam


In [16]:
config.get("S3", "LOG_DATA")

"'s3://udacity-dend/log-data'"

## Create clients for EC2, S3, IAM, and Redshift

In [20]:
ec2 = boto3.resource('ec2',
                       region_name           = REGION_NAME,
                       aws_access_key_id     = KEY,
                       aws_secret_access_key = SECRET
                    )

s3 = boto3.resource('s3',
                       region_name           = REGION_NAME,
                       aws_access_key_id     = KEY,
                       aws_secret_access_key = SECRET
                   )

iam = boto3.client('iam',aws_access_key_id = KEY,
                     aws_secret_access_key = SECRET,
                     region_name           = REGION_NAME
                  )

redshift = boto3.client('redshift',
                       region_name           = REGION_NAME,
                       aws_access_key_id     = KEY,
                       aws_secret_access_key = SECRET
                       )

## Check out the sample data sources on S3

In [21]:
sampleDbBucket =  s3.Bucket("awssampledbuswest2")
for obj in sampleDbBucket.objects.filter(Prefix="ssbgz"):
    print(obj)

s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/customer0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/dwdate.tbl.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0000_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0001_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0003_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0004_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0005_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0006_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0007_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='s

## STEP 1: IAM ROLE
- Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [22]:
from botocore.exceptions import ClientError

# 1.1 Create the role
try:
    print("1.1 Creating a new IAM Role") 
    dwhRole = iam.create_role(
        Path = '/',
        RoleName = DWH_IAM_ROLE_NAME ,
        Description = "Allows Redshift clusters to call AWS services on your behalf. DWH.",
        AssumeRolePolicyDocument = json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)

print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName = DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.1 Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dar1en-role-iam already exists.
1.3 Get the IAM role ARN
arn:aws:iam::787896866770:role/dar1en-role-iam


In [23]:
# 1.2 Attach Policy
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName = DWH_IAM_ROLE_NAME,
                       PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

1.2 Attaching Policy


200

In [24]:
# 1.3 Get and print the IAM role ARN
print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName = DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.3 Get the IAM role ARN
arn:aws:iam::787896866770:role/dar1en-role-iam


## STEP 2:  Redshift Cluster

- Create a RedShift Cluster
- For complete arguments to `create_cluster`, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster)

In [25]:
try:
    response = redshift.create_cluster(        
        #HW
        ClusterType   = DWH_CLUSTER_TYPE,
        NodeType      = DWH_NODE_TYPE,
        NumberOfNodes = int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName             = DWH_DB,
        ClusterIdentifier  = DWH_CLUSTER_IDENTIFIER,
        MasterUsername     = DWH_DB_USER,
        MasterUserPassword = DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles = [roleArn]  
    )
except Exception as e:
    print(e)

## 2.1 *Describe* the cluster to see its status
- run this block several times until the cluster status becomes `Available`

In [27]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data = x, columns = ["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier = DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dar1en-dwh-cluster-dev
1,NodeType,dc2.large
2,ClusterStatus,creating
3,MasterUsername,dev-dar1en
4,DBName,dar1en-redshift-dwh-db
5,VpcId,vpc-24bb175c
6,NumberOfNodes,4


<h2> 2.2 Take note of the cluster <font color='red'> endpoint and role ARN </font> </h2>

<font color='red'>DO NOT RUN THIS unless the cluster status becomes "Available" </font>

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

## STEP 3: Open an incoming  TCP port to access the cluster ednpoint

In [ ]:
try:
    vpc = ec2.Vpc(id = myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName  = defaultSg.group_name,
        CidrIp     = '0.0.0.0/0',
        IpProtocol = 'TCP',
        FromPort   = int(DWH_PORT),
        ToPort     = int(DWH_PORT)
    )
except Exception as e:
    print(e)

## STEP 4: Make sure you can connect to the clusterConnect to the cluster

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

## STEP 5: Clean up your resources

<b><font color='red'>DO NOT RUN THIS UNLESS YOU ARE SURE <br/> 
    </b>

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
iam.detach_role_policy(RoleName=DWH_IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)
#### CAREFUL!!